<a href="https://colab.research.google.com/github/rama96/pytorch_tutorials/blob/master/Quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pytroch Tutorials - Quickstart


## 1. Importing Library

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor


*   The torchvision.datasets module contains Dataset objects for many real-world vision data like CIFAR, COCO etc . 

*    Every TorchVision Dataset includes two arguments: 
transform and target_transform to modify the samples and labels respectively.

          1.   transform
          2.   target_transform




## 2.Loading Datasets

In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)


  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



## 3. Inspecting Datasets

In [9]:
print(len(training_data))

60000


In [13]:
type(training_data[0])

tuple

In [15]:
X0,y0 = training_data[0]

In [19]:
print("Shape of X0" , X0.shape )
print("y0" , y0  ," - of type" ,type(y0))

Shape of X0 torch.Size([1, 28, 28])
y0 9  - of type <class 'int'>


## 4. Creating Dataloaders using Datasets object

We pass the Dataset as an argument to DataLoader. This wraps an iterable over our dataset, and supports automatic batching, sampling, shuffling and multiprocess data loading. Here we define a batch size of 64, i.e. each element in the dataloader iterable will return a batch of 64 features and labels.

In [23]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in train_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


Converting datasets object into dataloader object does 2 things


1.   Makes it easier to pass these into a model so that we don't need to manually loop them according to batch size 
2.   Converts y from int to tensors .



In [30]:
import math

print(len(train_dataloader))
print(math.ceil(len(training_data)/64))

assert len(train_dataloader) == math.ceil(len(training_data)/64) 


938
938


## 5. Creating Models

In [38]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class MyOwnNeuralNetwork(nn.Module):
    def __init__(self):
        super(MyOwnNeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = MyOwnNeuralNetwork().to(device)
print(model)

Using cuda device
MyOwnNeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## 6. Selecting Loss Functions and Optimizer

In [48]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)


## 7. Defining Training and Testing functions

In [44]:
def train(dataloader, model, loss_fn, optimizer):
    
    # Total size of dataset for reference
    size = len(dataloader.dataset)
    
    # trains the model
    model.train()
    
    
    # Gives X , y for each batch
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error / loss
        # 1. Compute y_pred 
        # 2. Compute loss between y and y_pred using selectd loss function
        
        y_pred = model(X)
        loss = loss_fn(y_pred, y)

        # Backpropagation on optimizing for loss
        # 1. Sets gradients as 0 
        # 2. Compute the gradients using back_prop
        # 3. update the parameters using the gradients from step 2
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")




*   model.eval() is a kind of switch for some specific layers/parts of the model that behave differently during training and inference (evaluating) time. For example, Dropouts Layers, BatchNorm Layers etc. You need to turn off them during model evaluation, and .eval() will do it for you. In addition, the common practice for evaluating/validation is using torch.no_grad() in pair with model.eval() to turn off gradients computation:
*   List item



In [51]:
def test(dataloader, model, loss_fn):
    
    # Total size of dataset for reference
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    
    # Explanation given above
    model.eval()

    test_loss, correct = 0, 0
    with torch.no_grad():
        
        # Gives X , y for each batch
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    ## Calculating loss based on loss function defined
    test_loss /= num_batches
    
    ## Calculating Accuracy based on how many y match with y_pred
    correct /= size
    
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

## 8. Setting up the training loop

In [49]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")


Epoch 1
-------------------------------
loss: 2.300481  [    0/60000]
loss: 2.141749  [ 6400/60000]
loss: 1.774366  [12800/60000]
loss: 1.479828  [19200/60000]
loss: 1.116821  [25600/60000]
loss: 1.023198  [32000/60000]
loss: 0.982678  [38400/60000]
loss: 0.861220  [44800/60000]
loss: 0.858901  [51200/60000]
loss: 0.809449  [57600/60000]
Test Error: 
 Accuracy: 72.1%, Avg loss: 0.778661 

Epoch 2
-------------------------------
loss: 0.788091  [    0/60000]
loss: 0.830948  [ 6400/60000]
loss: 0.573801  [12800/60000]
loss: 0.774522  [19200/60000]
loss: 0.645073  [25600/60000]
loss: 0.628323  [32000/60000]
loss: 0.692384  [38400/60000]
loss: 0.682967  [44800/60000]
loss: 0.704456  [51200/60000]
loss: 0.648999  [57600/60000]
Test Error: 
 Accuracy: 78.0%, Avg loss: 0.628793 

Epoch 3
-------------------------------
loss: 0.574521  [    0/60000]
loss: 0.639828  [ 6400/60000]
loss: 0.434734  [12800/60000]
loss: 0.663636  [19200/60000]
loss: 0.573365  [25600/60000]
loss: 0.556045  [32000/600

## 9. Saving and loading back models

In [52]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")


Saved PyTorch Model State to model.pth


In [53]:
model = MyOwnNeuralNetwork()
model.load_state_dict(torch.load("model.pth"))


<All keys matched successfully>

## 10 Making custom predictions

In [55]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[1][0], test_data[1][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')


Predicted: "Pullover", Actual: "Pullover"
